In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"C:\Users\alex\Documents\repos\exess\smolkaa.github.io\api\base"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| output: false
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
T, m = 250.0, amu2kg(4) # [K], [kg] (mass of helium)
vd_mb = MaxwellBoltzmannVelocityDistribution(T, m)
vd_mbf = MaxwellBoltzmannFluxVelocityDistribution(T, m)

MaxwellBoltzmannFluxVelocityDistribution{Float64}(250.0, 6.642156158399789e-27)

In [3]:
velocity(vd_mb)

3-element Vector{Float64}:
 1291.3489313390633
 -952.8610817393512
  205.13179237252743

In [4]:
speed(vd_mb)

2259.7360766191214

In [5]:
azimuth(vd_mb)

1.4062035809721563

In [6]:
elevation(vd_mb)

-0.2664406854304329

In [7]:
zenith(vd_mb)

1.4452294396714378

In [8]:
#| output: false
v1, v2 = 100.0, 200.0   # speed [m s-1]
az1, az2 = -pi/4, pi/2  # azimuth [rad]
el1, el2 = pi/6, pi/4   # elevation [ad]
ze1, ze2 = pi/4, pi/3   # zenith [rad]

(0.7853981633974483, 1.0471975511965976)

In [9]:
speed_cdf(vd_mbf, v1, v2), azimuth_cdf(vd_mbf, az1, az2), elevation_cdf(vd_mbf, el1, el2), zenith_cdf(vd_mbf, ze1, ze2)

(0.0006760328420158297, 0.375, 0.25, 0.24999999999999992)

In [10]:
#| output: false
v1_mbf, v2_mbf = velocity(MaxwellBoltzmannFluxVelocityDistribution(T, m)), velocity(MaxwellBoltzmannFluxVelocityDistribution(T, m))
v1, v2 = speed(v1_mbf), speed(v2_mbf)
az1, az2 = azimuth(v1_mbf), azimuth(v2_mbf)
el1, el2 = elevation(v1_mbf), elevation(v2_mbf)
velocity_cdf(vd_mbf, v1_mbf, v2_mbf)

-0.008255912739929193

In [11]:
#| echo: false
abs(velocity_cdf(vd_mbf, v1_mbf, v2_mbf))

0.008255912739929193

In [12]:
abs(speed_cdf(vd_mbf, v1, v2)) * abs(azimuth_cdf(vd_mbf, az1, az2)) * abs(elevation_cdf(vd_mbf, el1, el2))

0.008255912739929192

In [13]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.MaxwellBoltzmannVelocityDistribution

```
[1] MaxwellBoltzmannVelocityDistribution{S<:AbstractFloat} <: AbstractVelocityDistribution{S}
[2] MaxwellBoltzmannVelocityDistribution(T::Real, m::Real)
```

Custom struct defining a (3D) Maxwell-Boltzmann velocity distribution. Uses the temperature `T` in Kelvin and the mass `m` in kg as inputs.


In [14]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.MaxwellBoltzmannFluxVelocityDistribution

```
[1] MaxwellBoltzmannFluxVelocityDistribution{S<:AbstractFloat} <: AbstractVelocityDistribution{S}
[2] MaxwellBoltzmannFluxVelocityDistribution(T::Real, m::Real)
```

Custom struct defining a (3D) Maxwell-Boltzmann flux velocity distribution. Uses the temperature     `T` in Kelvin and the mass `m` in kg as inputs.


In [15]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.azimuth

```
[1] azimuth([S::Type], vd::AbstractVelocityDistribution)
[2] azimuth([S::Type], v::Vector{<:Real})
[3] azimuth([S::Type], v::LocalCartesianVelocity)
```

[1] Draw a random azimuth angle (in [rad]) from the velocity distribution `vd`.

[2] & [3] Calculate the azimuth angle (in [rad]) of the velocity vector `v` (in [m s-1]).  Alternatively, the azimuth angle can be calculated from a `LocalCartesianVelocity` `v` (in [m s-1]).


In [16]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.azimuth_cdf

```
[1] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, theta::Tuple{Real, Real})
[2] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, theta1::Real, theta2::Real)
[3] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, theta::Real)
[4] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, v1::AbstractVector, v2::AbstractVector)
[5] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[6] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, v1::LocalCartesianVelocity, v2::LocalCartesianVelocity)
[7] azimuth_cdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Computes the cumulative distribution function of the azimuth angle of the velocity distribution `vd`, between the azimuth angles `theta[1]`/`theta1` and `theta[2]`/`theta2` (in [rad]). Should only one angle be given, the lower boundary is assumed to be lower boundary of the respective domain, `-pi`.

If instead of an azimuth angle `theta` a velocity vector `v` (in [m s-1]) or a `LocalCartesianVelocity` `v` (in [m s-1]) is given, the azimuth angle is calculated from the velocity vector and the cumulative distribution function is calculated accordingly.


In [17]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.azimuth_pdf

```
[1] azimuth_pdf([S::Type], vd::AbstractVelocityDistribution, theta::Real)
[2] azimuth_pdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[3] azimuth_pdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Calculates the probability density of the azimuth angle of the velocity distribution `vd`, at the azimuth angle `theta` (in [rad]). Alternatively, the probability density can be calculated at a velocity vector `v` (in [m s-1]) or a `LocalCartesianVelocity` `v` (in [m s-1]). Note that for  [2] the vector assumes a local cartesian coordinate system (see `LocalCartesianVelocity`).


In [18]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.elevation

```
[1] elevation([S::Type], vd::AbstractVelocityDistribution)
[2] elevation([S::Type], v::AbstractVector)
[3] elevation([S::Type], v::LocalCartesianVelocity)
```

[1] Draw a random elevation angle (in [rad]) from the velocity distribution `vd`.

[2] & [3] Calculate the elevation angle (in [rad]) of the velocity vector `v` (in [m s-1]). Alternatively, the elevation angle can be calculated from a `LocalCartesianVelocity` `v` (in [m s-1]).

**Note**

  * Domain of `MaxwellBoltzmannVelocityDistribution` is `[-pi/2, pi/2]`
  * Domain of `MaxwellBoltzmannFluxVelocityDistribution` is `[0, pi/2]`


In [19]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.elevation_cdf

```
[1] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, phi::Tuple{Real, Real})
[2] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, phi1::Real, phi2::Real)
[3] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, phi::Real)
[4] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, v1::AbstractVector, v2::AbstractVector)
[5] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[6] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, v1::LocalCartesianVelocity, v2::LocalCartesianVelocity)
[7] elevation_cdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Computes the cumulative distribution function of the elevation angle of the velocity distribution `vd`, between the elevation angles `phi[1]`/`phi1` and `phi[2]`/`phi2` (in [rad]). Should only one angle be given, the lower boundary is assumed to be lower boundary of the respective domain. 

If instead of an elevation angle `theta` a velocity vector `v` (in [m s-1]) or a `LocalCartesianVelocity` `v` (in [m s-1]) is given, the elevation angle is calculated from the velocity vector and the cumulative distribution function is calculated accordingly.

** Notes**

  * Domain of `MaxwellBoltzmannVelocityDistribution` is `[-pi/2, pi/2]`, thus, using [3,5,7], the lower boundary is `-pi/2`.
  * Domain of `MaxwellBoltzmannFluxVelocityDistribution` is `[0, pi/2]`, thus, using [3,5,7], the lower boundary is `0`.


In [20]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.elevation_pdf

```
[1] elevation_pdf([S::Type], vd::AbstractVelocityDistribution, phi::Real)
[2] elevation_pdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[3] elevation_pdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Calculates the probability density of the elevation angle of the velocity distribution `vd`, at the elevation angle `phi` (in [rad]). Alternatively, the probability density can be calculated at a velocity vector `v` (in [m s-1]) or a `LocalCartesianVelocity` `v` (in [m s-1]). Note that for  [2] the vector assumes a local cartesian coordinate system (see `LocalCartesianVelocity`).


In [21]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.speed

```
[1] speed([S::Type], vd::AbstractVelocityDistribution)
[2] speed([S::Type], v::AbstractVector)
[3] speed([S::Type], v::LocalCartesianVelocity)
[4] speed([S::Type], vd::MaxwellBoltzmannVelocityDistribution, mode::Symbol)
```

[1] Draw a random speed (in [m s-1]) from the velocity distribution `vd`.

[2] & [3] Calculate the speed (in [m s-1]) of the velocity vector `v` (in [m s-1]). Alternatively, the speed can be calculated from a `LocalCartesianVelocity` `v` (in [m s-1]).

[4] Calculate the typical speed (in [m s-1]) of the velocity distribution `vd` based on the mode `mode`. The following modes are available:

  * `:prob` for most probable velocity
  * `:mean` for mean velocity
  * `:rms` for root-mean-square velocity


In [22]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.speed_cdf

```
[1] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v::Tuple{Real, Real}; N=1000)
[2] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v::Tuple{Integer, Integer}; kwargs...)
[3] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v::Real; kwargs...)
[4] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v1::AbstractVector, v2::AbstractVector)
[5] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[6] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v1::LocalCartesianVelocity, v2::LocalCartesianVelocity)
[7] speed_cdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Computes the cumulative distribution function of the speed of the velocity distribution `vd` based on numerical integration of the respective probability density function between the upper and lower speed given in `v` (in [m s-1]) (if `v` is scalar, the lower boundary is zero).

Alternatively, the cumulative distribution function can be calculated between the upper and lower speed given in `v` (in [m s-1]), either as an `AbstractVector` or a `LocalCartesianVelocity` (in [m s-1]). Note that [4] and [5] expect the vector to be given in cartesian coordinates (see `LocalCartesianVelocity`).


In [23]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.speed_pdf

```
[1] speed_pdf([S::Type], vd::AbstractVelocityDistribution, v::Real)
[2] speed_pdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[3] speed_pdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Calculates the probability density of the speed of the velocity distribution `vd`, at the speed `v` (in [m s-1]). 


In [24]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.velocity

```
[1] velocity([S::Type], vd::AbstractVelocityDistribution)
```

Draw a random, trhee-dimensional velocity vector (in [m s-1]) from the velocity distribution `vd`.

**Notes**

  * The three-dimensional vector is given in local cartesian coordinates (see `LocalCartesianVelocity`). This means that `v[1]` points locally east, `v[2]` points locally north, and `v[3]` points locally up.


In [25]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.velocity_cdf

```
[1] velocity_cdf([S::Type], vd::AbstractVelocityDistribution, v1::AbstractVector, v2::AbstractVector)
[2] velocity_cdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[3] velocity_cdf([S::Type], vd::AbstractVelocityDistribution, v1::LocalCartesianVelocity, v2::LocalCartesianVelocity)
[4] velocity_cdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Computes the cumulative distribution function of the velocity vector of the velocity distribution `vd`, between the velocity vectors `v1` and `v2` (in [m s-1]). Should only one vector be given, the lower boundary is assumed to be the zero vector.

Note that if an `AbstractVector` is given, it is assumed to be given in cartesian coordinates (see `LocalCartesianVelocity`).


In [26]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.velocity_pdf

```
[1] velocity_pdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[2] velocity_pdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Calculates the probability density of the velocity vector `v` (in [m s-1]) of the velocity distribution `vd`.


In [27]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.zenith

```
[1] zenith([S::Type], vd::AbstractVelocityDistribution)
[2] zenith([S::Type], v::Vector{<:Real})
[3] zenith([S::Type], v::LocalCartesianVelocity)
```

[1] Draw a random zenith angle (in [rad]) from the velocity distribution `vd`.

[2] & [3] Calculate the zenith angle (in [rad]) of the velocity vector `v` (in [m s-1]). Alternatively,  the zenith angle can be calculated from a `LocalCartesianVelocity` `v` (in [m s-1]).

**Notes**

  * the zenith angle is the same-sign pi/2-inversion of the elevation angle


In [28]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.zenith_cdf

```
[1] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, psi::Tuple{Real, Real})
[2] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, psi1::Real, psi2::Real)
[3] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, psi::Real)
[4] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, v1::AbstractVector, v2::AbstractVector)
[5] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[6] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, v1::LocalCartesianVelocity, v2::LocalCartesianVelocity)
[7] zenith_cdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Computes the cumulative distribution function of the zenith angle of the velocity distribution `vd`, between the zenith angles `psi[1]`/`psi1` and `psi[2]`/`psi2` (in [rad]). Should only one angle be given, the lower boundary is assumed to be lower boundary of the respective domain.

** Notes**

  * Domain of `MaxwellBoltzmannVelocityDistribution` is `[-pi/2, pi/2]`, thus, using [3,5,7], the lower boundary is `-pi/2`.
  * Domain of `MaxwellBoltzmannFluxVelocityDistribution` is `[0, pi/2]`, thus, using [3,5,7], the lower boundary is `0`.


In [29]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.zenith_pdf

```
[1] zenith_pdf([S::Type], vd::AbstractVelocityDistribution, psi::Real)
[2] zenith_pdf([S::Type], vd::AbstractVelocityDistribution, v::AbstractVector)
[3] zenith_pdf([S::Type], vd::AbstractVelocityDistribution, v::LocalCartesianVelocity)
```

Calculates the probability density of the zenith angle of the velocity distribution `vd`, at the zenith angle `psi` (in [rad]). Alternatively, the probability density can be calculated at a velocity vector `v` (in [m s-1]) or a `LocalCartesianVelocity` `v` (in [m s-1]). Note that for  [2] the vector assumes a local cartesian coordinate system (see `LocalCartesianVelocity`).
